In [1]:
import pandas as pd
import numpy as np

In [2]:
daily_data = pd.read_csv('cleaned data/daily_data.csv')
daily_data['date'] = pd.to_datetime(daily_data['date'])
daily_data = daily_data.set_index('date')
daily_data.head()

,ECB_assets,FED_assets,EUR/USD,VIX,fed_target,Euribor_90d,libor3M,surprise_index,cip,sov_diff2Y,Meeting_ECB,Meeting_FED,ecb_fdr
date,,,,,,,,,,,,,
2008-01-01,254025.0,890662.0,1.4587,22.50,4.25,4.4815,3.898750,NaN,102.233268,1.166987,0.0,0.0,3.0
2008-01-02,254025.0,922187.0,1.4725,24.02,4.25,4.4815,3.095000,-0.204000,86.849101,1.156674,0.0,0.0,3.0
2008-01-03,254025.0,922187.0,1.4746,24.02,4.25,4.4815,2.889375,-0.160333,88.964206,1.196361,0.0,0.0,3.0
2008-01-04,254025.0,922187.0,1.4743,22.68,4.25,4.4815,2.683750,-0.116667,86.474503,1.276048,0.0,0.0,3.0
2008-01-05,254025.0,922187.0,1.4743,18.88,4.25,4.4815,2.784380,-0.073000,83.054214,1.259069,0.0,0.0,3.0


In [3]:
# Resample Meeting_ECB and Meeting_FED using max
monthly_data = pd.DataFrame()
monthly_data['Meeting_ECB'] = daily_data['Meeting_ECB'].resample('M').max()
monthly_data['Meeting_FED'] = daily_data['Meeting_FED'].resample('M').max()

# Resample other columns using mean
for col in daily_data.columns:
    if col not in ['Meeting_ECB', 'Meeting_FED']:
        monthly_data[col] = daily_data[col].resample('M').mean()

#monthly_data = daily_data.dropna()

C:\Users\mathi\AppData\Local\Temp\ipykernel_1660\3949876541.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data['Meeting_ECB'] = daily_data['Meeting_ECB'].resample('M').max()
C:\Users\mathi\AppData\Local\Temp\ipykernel_1660\3949876541.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data['Meeting_FED'] = daily_data['Meeting_FED'].resample('M').max()
C:\Users\mathi\AppData\Local\Temp\ipykernel_1660\3949876541.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data[col] = daily_data[col].resample('M').mean()
C:\Users\mathi\AppData\Local\Temp\ipykernel_1660\3949876541.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data[col] = daily_data[col].resample('M').mean()
C:\Users\mathi\AppData\Local\Temp\ipykernel_1660\3949876541.py:9: 

In [4]:
monthly_data['BS'] = np.log(monthly_data['ECB_assets'] / monthly_data['FED_assets'])
monthly_data.drop(['ECB_assets', 'FED_assets'], axis=1, inplace=True)

monthly_data['Money_mkt3M'] = monthly_data['Euribor_90d'] - monthly_data['libor3M']
monthly_data.drop(['Euribor_90d', 'libor3M'], axis=1, inplace=True)

monthly_data['d_rate'] = monthly_data['ecb_fdr'] - monthly_data['fed_target']
monthly_data.drop(['ecb_fdr', 'fed_target'], axis=1, inplace=True)
print(monthly_data.head())

            Meeting_ECB  Meeting_FED   EUR/USD        VIX  surprise_index  \
date                                                                        
2008-01-31          0.0          0.0  1.471110  28.595484       -0.125298   
2008-02-29          0.0          0.0  1.474321  26.654828       -0.292243   
2008-03-31          0.0          0.0  1.551531  28.710645       -0.317323   
2008-04-30          0.0          0.0  1.576020  24.573833       -0.287636   
2008-05-31          0.0          0.0  1.555931  22.350645       -0.159774   

                  cip  sov_diff2Y        BS  Money_mkt    d_rate  
date                                                              
2008-01-31  69.326813    1.420358 -1.254592   1.138384 -0.975806  
2008-02-29  58.556804    1.577859 -1.144487   1.240105  0.000000  
2008-03-31  87.863301    1.865197 -1.154450   1.759585  0.338710  
2008-04-30  89.958693    1.656922 -1.126674   1.913197  0.758333  
2008-05-31  43.121501    1.557729 -1.090163   2.144850  1.

In [5]:
monthly_data.to_csv('cleaned data/monthly_data.csv')